In [11]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from scipy.stats import norm
from scipy.optimize import minimize

In [5]:
# define true parameters
I = 100
J = 2
T = 10
beta = -0.1
price_loc = 2
price_scale = 0.5
alpha_1 = 1
alpha_2 = 1

In [4]:
# supply side data 
# t X price_1 X price_2
time_frame = np.array(range(1, T+1))
price_1 = np.random.normal(price_loc, price_scale, T)
price_2 = np.random.normal(price_loc, price_scale, T)
df_supply_side = df({"t": time_frame, "price_1": price_1, "price_2": price_2})

In [6]:
# individual side data
# i X t 
individual_frame = []
time_frame = []

for i in range(1, I+1):
    for t in range(1, T+1):
        individual_frame.append(i)
        time_frame.append(t)

df_individual_side = df({"i": individual_frame, "t": time_frame})

In [8]:
# discrete choice data
# i X t X price_1 X price_2 X buy_product_1 X buy_product_2
# (1) pre choice decision
df_discrete_choice = pd.merge(df_individual_side, df_supply_side, left_on="t", right_on="t")

# (2) attach the choice decision
util_product_1 = alpha_1 + beta * df_discrete_choice['price_1'].values 
util_product_2 = alpha_2 + beta * df_discrete_choice['price_2'].values 

prob_product_1 = np.exp(util_product_1)/(1+np.exp(util_product_1)+np.exp(util_product_2))
prob_product_2 = np.exp(util_product_2)/(1+np.exp(util_product_1)+np.exp(util_product_2))
prob_outer = 1 - prob_product_1 - prob_product_2

### decide the product decision (let the error term choose the product)
random_u = np.random.uniform(low=0, high=1, size= I*T)

d_0 = (random_u <= prob_outer).astype(int)
d_1 = ((random_u <= prob_outer + prob_product_1) * (random_u > prob_outer)).astype(int)
d_2 = (random_u > prob_outer + prob_product_1).astype(int)

df_discrete_choice['d_0'] = d_0
df_discrete_choice['d_1'] = d_1
df_discrete_choice['d_2'] = d_2

## 1. MLE

In [10]:
def llh(theta):
    alpha_1, alpha_2, beta = theta

    util_product_1 = alpha_1 + beta * df_discrete_choice['price_1'].values 
    util_product_2 = alpha_2 + beta * df_discrete_choice['price_2'].values 

    prob_product_1 = np.exp(util_product_1)/(1+np.exp(util_product_1)+np.exp(util_product_2))
    prob_product_2 = np.exp(util_product_2)/(1+np.exp(util_product_1)+np.exp(util_product_2))
    prob_outer = 1 - prob_product_1 - prob_product_2

    lh_array = df_discrete_choice['d_0'].values * prob_outer + df_discrete_choice['d_1'].values * prob_product_1 + df_discrete_choice['d_2'].values * prob_product_2
    llh_array = np.log(lh_array)
    llh = llh_array.sum()

    return -llh

In [17]:
res = minimize(llh, [0.5, 0.5, -0.5], method="Nelder-Mead")
res.x

array([ 0.97466083,  0.88122269, -0.09637773])

## 2. Bayesian Estimation

In [18]:
beta_init = -0.5
alpha_1_init = 0.5
alpha_2_init = 0.5

In [19]:
def llh2(alpha_1, alpha_2, beta):
    util_product_1 = alpha_1 + beta * df_discrete_choice['price_1'].values 
    util_product_2 = alpha_2 + beta * df_discrete_choice['price_2'].values 

    prob_product_1 = np.exp(util_product_1)/(1+np.exp(util_product_1)+np.exp(util_product_2))
    prob_product_2 = np.exp(util_product_2)/(1+np.exp(util_product_1)+np.exp(util_product_2))
    prob_outer = 1 - prob_product_1 - prob_product_2

    lh_array = df_discrete_choice['d_0'].values * prob_outer + df_discrete_choice['d_1'].values * prob_product_1 + df_discrete_choice['d_2'].values * prob_product_2
    llh_array = np.log(lh_array)
    llh = llh_array.sum()
    return llh

In [22]:
beta_init = -0.5
alpha_1_init = 0.5
alpha_2_init = 0.5

counter = 0

alpha_1_history = []
alpha_2_history = []
beta_history = []

while (counter <= 3000):
    # 1. draw alpha_1
    alpha_1_candidate = alpha_1_init + np.random.normal(loc=0, scale=0.1, size=1)[0]
    l_accept_rate = min(0, llh2(alpha_1_candidate, alpha_2_init, beta_init)-llh2(alpha_1_init, alpha_2_init, beta_init))
    u = np.random.uniform(low=0, high=1, size=1)[0]
    ln_u = np.log(u)
    if ln_u <= l_accept_rate:
        alpha_1_post = alpha_1_candidate
    else:
        alpha_1_post = alpha_1_init

    # 2. draw alpha_2
    alpha_2_candidate = alpha_2_init + np.random.normal(loc=0, scale=0.1, size=1)[0]
    l_accept_rate = min(0, llh2(alpha_1_post, alpha_2_candidate, beta_init)-llh2(alpha_1_post, alpha_2_init, beta_init))
    u = np.random.uniform(low=0, high=1, size=1)[0]
    ln_u = np.log(u)
    if ln_u <= l_accept_rate:
        alpha_2_post = alpha_2_candidate
    else:
        alpha_2_post = alpha_2_init
    
    # 3. draw beta
    beta_candidate = beta_init + np.random.normal(loc=0, scale=0.1, size=1)[0]
    l_accept_rate = min(0, llh2(alpha_1_post, alpha_2_post, beta_candidate)-llh2(alpha_1_post, alpha_2_post, beta_init))
    u = np.random.uniform(low=0, high=1, size=1)[0]
    ln_u = np.log(u)
    if ln_u <= l_accept_rate:
        beta_post = beta_candidate
    else:
        beta_post = beta_init

    # save and update
    alpha_1_history.append(alpha_1_post)
    alpha_2_history.append(alpha_2_post)
    beta_history.append(beta_post)

    alpha_1_init = alpha_1_post
    alpha_2_init = alpha_2_post
    beta_init = beta_post

    counter += 1

In [25]:
np.array(alpha_1_history[500:]).mean()

0.9800208380822656

In [26]:
np.array(alpha_2_history[500:]).mean()

0.8860651050309422

In [27]:
np.array(beta_history[500:]).mean()

-0.09927396319392563